In [2]:
import os

In [33]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

Clone github repo

In [2]:
import requests
from langchain.document_loaders import TextLoader


In [34]:
url = "https://www.legalandgeneral.com/retirement/our-products/"

In [35]:
res = requests.get(url)

In [37]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, "html.parser")
text = soup.get_text()

In [38]:
text

"\n\nOur Retirement Products | Legal & General\n\n\n\n\n\nPersonal Adviser Employer Institutional Menu Investments  Pensions & Retirement  Insurance  Existing customers  Search Log inRegisterContact usSearch Search term  Search Log inRegisterContact us Back Investments  Stocks and Shares ISA  Our services  ISA allowance  ISAs explained  Understanding risk  Resources and articles  A Little Bit Richer podcast  Cash ISA vs Stocks and Shares ISA  What’s the best ISA for me?  Investing for beginners  Types of ISA  How to protect savings from inflation  Transferring your ISA  Contact us  Back Pensions & Retirement  Saving  Back Saving  Saving for retirement  Personal Pension  Pension transfers and consolidation  Tax benefits  Understanding risk  Workplace Pensions  Pensions explained  Planning tools and calculators  Planning & At Retirement  Back Planning & At Retirement  Planning for retirement  Retirement advice service  Deciding how to use your pension  Ways to take your pension  Pension 

In [43]:
lines = text.split('\n')
non_empty_lines = [line for line in lines if line.strip()]
cleaned_text = '\n'.join(non_empty_lines)

In [44]:
cleaned_text

"Our Retirement Products | Legal & General\nPersonal Adviser Employer Institutional Menu Investments  Pensions & Retirement  Insurance  Existing customers  Search Log inRegisterContact usSearch Search term  Search Log inRegisterContact us Back Investments  Stocks and Shares ISA  Our services  ISA allowance  ISAs explained  Understanding risk  Resources and articles  A Little Bit Richer podcast  Cash ISA vs Stocks and Shares ISA  What’s the best ISA for me?  Investing for beginners  Types of ISA  How to protect savings from inflation  Transferring your ISA  Contact us  Back Pensions & Retirement  Saving  Back Saving  Saving for retirement  Personal Pension  Pension transfers and consolidation  Tax benefits  Understanding risk  Workplace Pensions  Pensions explained  Planning tools and calculators  Planning & At Retirement  Back Planning & At Retirement  Planning for retirement  Retirement advice service  Deciding how to use your pension  Ways to take your pension  Pension Annuity  Pensi

In [45]:
with open("insurance_products.txt", "w") as f:
    f.write(cleaned_text)

In [6]:
# loader = TextLoader('insurance_products.txt')

loader = TextLoader("info")

In [7]:
documents = loader.load()

RuntimeError: Error loading info

### Chunkings ###

In [49]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [50]:
texts = documents_splitter.split_documents(documents)

In [52]:
texts

[Document(page_content='Our Retirement Products | Legal & General', metadata={'source': 'insurance_products.txt'}),
 Document(page_content='Personal Adviser Employer Institutional Menu Investments  Pensions & Retirement  Insurance  Existing customers  Search Log inRegisterContact usSearch Search term  Search Log inRegisterContact us Back Investments  Stocks and Shares ISA  Our services  ISA allowance  ISAs explained  Understanding risk  Resources and articles  A Little Bit Richer podcast  Cash ISA vs Stocks and Shares ISA  What’s the best ISA for me?  Investing for beginners  Types of ISA  How to protect savings from inflation  Transferring your ISA  Contact us  Back Pensions & Retirement  Saving  Back Saving  Saving for retirement  Personal Pension  Pension transfers and consolidation  Tax benefits  Understanding risk  Workplace Pensions  Pensions explained  Planning tools and calculators  Planning & At Retirement  Back Planning & At Retirement  Planning for retirement  Retirement adv

In [51]:
len(texts)

6

### Embedding Model

In [53]:
import os
os.environ["OPENAI_API_KEY"] = "*****************************"

In [54]:
embeddings=OpenAIEmbeddings(disallowed_special=())

### Knowledge Base (vector DB) ###

In [55]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

#### LLM Wrapper

In [56]:
llm = ChatOpenAI()

In [57]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [58]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

Q & A

In [59]:
question = "which are the retirement plans available?"

In [61]:
question2 = "what is the capital of India"

In [62]:
result = qa(question2)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


I don't know the answer to that question.
